In [1]:
import pandas as pd
import numpy as np
import xml.etree.ElementTree as ET
from pymongo import MongoClient

import os

import downloadfilesfromTCE as tce

In [2]:
client = MongoClient('localhost',27017)
db = client['db_data_gov']

def save_mongo(dataframe, collection_name=None):
    result = db[collection_name].insert_many(dataframe.to_dict(orient='records'))
    return result

def find_cdIBGE(nmMunicipio = []):
    query = {'municipio' : { '$in': nmMunicipio } }
    projection = { '_id' : 0, 'cdIBGE' : 1}
    result = db.municipio.find(query, projection)
    return result
    

### 1) Obtendo os dados do site Transparencia Para Totos

In [3]:
anos = ['2013', '2014', '2015', '2016', '2017']
municipios = [
    'Londrina'
]

In [4]:
def obterCodigoMunicipio(nomesMunicipios = []) :
    queryResult = find_cdIBGE(municipios)
    m = map(lambda x : x['cdIBGE'], queryResult)
    return list(m)


In [5]:
codMunicipios = obterCodigoMunicipio(municipios)

for ano in anos :
    for cod in codMunicipios :
#         tce.get_file(cod,ano,'licitacao')
        print(cod,ano,'licitacao')

4113700 2013 licitacao
4113700 2013 licitacao
4113700 2014 licitacao
4113700 2014 licitacao
4113700 2015 licitacao
4113700 2015 licitacao
4113700 2016 licitacao
4113700 2016 licitacao
4113700 2017 licitacao
4113700 2017 licitacao


In [ ]:
tce.extract_zip_to('licitacao')

In [6]:
diretorio = "data/licitacao/"
files = os.listdir(diretorio)
files = list(map(lambda f : diretorio + f, files))

len(files)

30

### 2) Parse dos dado e salvando no MongoDB

In [24]:
def fromXMLtoDataFrame(filename):
    xml = ET.parse(filename)
    root = xml.getroot()
    data = []
    count = 0
    for child in root :
        data.append(child.attrib)
        count += 1
    df = pd.DataFrame(data)
    return df, count

In [8]:
'''Função responsável por garantir as conversões '''
def parserFromLicitacao(df_licitacao):
    casting_dcit = {
        'vlLicitacao' : 'float',
        'dtAbertura' : 'datetime64[ns]',
        'dtEdital' : 'datetime64[ns]',
        'dtEnvio' : 'datetime64[ns]',
    }
    df_licitacao.astype(casting_dcit,copy=False)
    
    return df_licitacao
    

In [9]:
def parserFromLicitacaoParticipante(df_participante):
    return df_participante

In [10]:
def parserFromLicitacaoVencedor(df_vencedor):
    casting = {
        'dtHomologacao' : 'datetime64[ns]',
        'dtPrazoEntregaPropostaLicitacao' : 'datetime64[ns]',
        'dtValidadeProposta' : 'datetime64[ns]',
        'nrQuantidade' : 'float',
        'nrQuantidadePropostaLicitacao' : 'float',
        'nrQuantidadeVencedorLicitacao' : 'float',
        'vlLicitacaoVencedorLicitacao' : 'float',
        'vlMaximoTotal' : 'float',
        'vlMaximoUnitarioitem' : 'float',
        'vlMinimoTotal' : 'float',
        'vlMinimoUnitarioItem' : 'float',
        'vlPropostaItem' : 'float'
    }
    return df_vencedor.astype(casting,copy=False)

In [18]:
files_config = {
    'Licitacao.xml' : {
        'parser' : parserFromLicitacao,
        'collection' : 'licitacao'
    },
    'LicitacaoVencedor.xml' : {
        'parser' : parserFromLicitacaoVencedor,
        'collection' : 'licitacaoVencedor'
    },
    'LicitacaoParticipante.xml' : {
        'parser' : parserFromLicitacaoParticipante,
        'collection' : 'licitacaoParticipante '
    }
}

def determine_file_type(file, f_config=files_config):
    index = file.rfind('_')
    if index < 0 : 
        raise Error('Key error')
    else :
        index += 1
        f_type = file[index:]
        return f_config[f_type]

def files_handler(files = []):
    verificar = []
    for filename in files :
        config = determine_file_type(filename)
        parser = config['parser']
        try:
            df,qtd = fromXMLtoDataFrame(filename)
#             df = parser(df)
#             result = save_mongo(df, collection_name=config['collection'])
#             if qtd != len(result.inserted_ids):
#                 verificar.append(verificar)
        except Exception as ERR:
            print(ERR)
            print('file name: ',filename)
    if verificar :
        return verificar
    else :
        return True
        

In [19]:
files_type = list(files_config.keys())
files_type

['Licitacao.xml', 'LicitacaoVencedor.xml', 'LicitacaoParticipante.xml']

In [20]:
files[20]

'data/licitacao/2016_411370_LicitacaoVencedor.xml'

In [25]:
files_handler(files)

reference to invalid character number: line 1, column 10148122
file name: 
data/licitacao/2015_411370_LicitacaoVencedor.xml
reference to invalid character number: line 1, column 6494945
file name: 
data/licitacao/2017_411990_LicitacaoVencedor.xml


True